<a href="https://colab.research.google.com/github/narfdf999/Arquitectura/blob/master/PMV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Primera aproximación al problema de generar edificio a lo largo de un camino dibujado por el arquitecto***

## Librerías

---



In [0]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model ,Sequential, load_model
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

## Visualización y preprocesamiento de datos

---



### Prueba con fichero 17_a

In [32]:
direcciones_v=pd.read_csv('17_a_direcciones.csv',sep=',')
print(direcciones_v)
print(direcciones_v.shape)
direcciones=np.genfromtxt('17_a_direcciones.csv',delimiter=',')
print(direcciones)
print(direcciones.shape)
puntos_v=pd.read_csv('17_a_puntos.csv',sep=',')
print(puntos_v)
print(puntos_v.shape)
puntos=np.genfromtxt('17_a_puntos.csv',delimiter=',')
print(puntos)
print(puntos.shape)

      -0.923764078523375  ...  -0.94776374744804.59
0              -0.923764  ...             -0.947764
1              -0.923764  ...             -0.947764
2              -0.923764  ...             -0.947764
3              -0.923764  ...             -0.947764
4              -0.923764  ...             -0.947764
...                  ...  ...                   ...
1013           -0.937844  ...             -0.934182
1014           -0.937844  ...             -0.934182
1015           -0.937844  ...             -0.934182
1016           -0.937844  ...             -0.934182
1017           -0.937844  ...             -0.934182

[1018 rows x 1259 columns]
(1018, 1259)
[[-0.92376408 -0.91654703 -0.90399378 ... -0.9401907  -0.94578498
  -0.94776375]
 [-0.92376408 -0.91654703 -0.90399378 ... -0.9401907  -0.94578498
  -0.94776375]
 [-0.92376408 -0.91654703 -0.90399378 ... -0.9401907  -0.94578498
  -0.94776375]
 ...
 [-0.93784381 -0.93195966 -0.92172482 ... -0.92463978 -0.93168862
  -0.93418188]
 [-0.9

### Matriz de ruido aleatorio de forma (1019,128)

In [33]:
v_aleatorio=np.random.random((1019,128))
print(v_aleatorio)
print(v_aleatorio.shape)

[[0.67801569 0.3567908  0.36855205 ... 0.32237528 0.02188026 0.11358782]
 [0.71173648 0.05563133 0.56866932 ... 0.47131149 0.33742512 0.81307473]
 [0.41679122 0.12675047 0.60381848 ... 0.11115439 0.82221191 0.78666747]
 ...
 [0.31866231 0.82369266 0.77052349 ... 0.63506795 0.55723437 0.67523174]
 [0.14700239 0.90103154 0.8510812  ... 0.11928675 0.18576142 0.3188208 ]
 [0.09437547 0.63810211 0.21186771 ... 0.74848457 0.61466761 0.75740824]]
(1019, 128)


### Concatenación de ruido con puntos para la entrada de la GAN

In [34]:
gen_input=np.concatenate([v_aleatorio,puntos], axis=1)
print(gen_input)
print(gen_input.shape)

[[ 6.78015687e-01  3.56790799e-01  3.68552054e-01 ...  9.21602370e-01
  -5.85746679e-01  0.00000000e+00]
 [ 7.11736477e-01  5.56313316e-02  5.68669320e-01 ...  9.21836604e-01
  -5.78241624e-01  9.81354269e-04]
 [ 4.16791223e-01  1.26750474e-01  6.03818479e-01 ...  9.22070869e-01
  -5.70735582e-01  1.96270854e-03]
 ...
 [ 3.18662305e-01  8.23692658e-01  7.70523489e-01 ...  9.45310318e-01
   1.73875582e-01  9.97055937e-01]
 [ 1.47002387e-01  9.01031536e-01  8.51081198e-01 ...  9.45463455e-01
   1.78782228e-01  9.98037291e-01]
 [ 9.43754727e-02  6.38102112e-01  2.11867714e-01 ...  9.45615168e-01
   1.83643243e-01  9.99018646e-01]]
(1019, 132)


## Implementación de la GAN

---



Optimizador ADAM


In [0]:
def adam_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

Generador

In [37]:
def create_generator():
  generator=Sequential()
  generator.add(Dense(units=256,input_dim=132))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(units=512))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(units=1024))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(units=1259, activation='tanh'))

  generator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
  return generator

g=create_generator()
g.summary()
  



Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_122 (Dense)            (None, 256)               34048     
_________________________________________________________________
leaky_re_lu_100 (LeakyReLU)  (None, 256)               0         
_________________________________________________________________
dense_123 (Dense)            (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_101 (LeakyReLU)  (None, 512)               0         
_________________________________________________________________
dense_124 (Dense)            (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_102 (LeakyReLU)  (None, 1024)              0         
_________________________________________________________________
dense_125 (Dense)            (None, 1259)            

Discriminador

In [38]:
def create_discriminator():
    discriminator=Sequential()
    discriminator.add(Dense(units=1024,input_dim=1263))
    discriminator.add(LeakyReLU(0.2))
       
    discriminator.add(Dense(units=512))
    discriminator.add(LeakyReLU(0.2))
       
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dense(units=128))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dense(units=64))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dense(units=32))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(units=1, activation='softmax'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return discriminator
d =create_discriminator()
d.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_126 (Dense)            (None, 1024)              1294336   
_________________________________________________________________
leaky_re_lu_103 (LeakyReLU)  (None, 1024)              0         
_________________________________________________________________
dense_127 (Dense)            (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_104 (LeakyReLU)  (None, 512)               0         
_________________________________________________________________
dense_128 (Dense)            (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_105 (LeakyReLU)  (None, 256)               0         
_________________________________________________________________
dense_129 (Dense)            (None, 128)             

GAN

In [41]:
def create_gan(discriminator, generator):
  discriminator.trainable=False
  gan_input = Input(shape=(132,))
  x = generator(gan_input)
  p = Input(shape=(4,))
  x = keras.layers.Concatenate()([x,p])
  gan_output=discriminator(x)
  gan = Model(inputs=[gan_input, p], outputs=gan_output)
  gan.compile(loss='binary_crossentropy', optimizer='adam')
  return gan

gan=create_gan(d,g)
gan.summary()

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           (None, 132)          0                                            
__________________________________________________________________________________________________
sequential_23 (Sequential)      (None, 1259)         1981419     input_27[0][0]                   
__________________________________________________________________________________________________
input_28 (InputLayer)           (None, 4)            0                                            
__________________________________________________________________________________________________
concatenate_14 (Concatenate)    (None, 1263)         0           sequential_23[2][0]              
                                                                 input_28[0][0]            

## Entrenamiento

In [53]:
def training(epochs=1, batch_size=1019):
    
    # Cargando datos
    (gen_input, direcciones, puntos)=load_data()

    # Creando GAN
    generator = create_generator()
    discriminator = create_discriminator()
    gan = create_gan(discriminator, generator)
    
    for e in range(1,epochs+1 ):
        print("Epoch %d" %e)
        for _ in tqdm(range(batch_size)):
            # Generando imagenes falsas sin puntos
            generated_images = generator.predict(gen_input)
            
            # Concatenando imagenes generadas con sus puntos
            generated_images = np.concatenate([generated_images, puntos], axis=1)

            # Concatenar imagenes reales con sus puntos
            image_batch = np.concatenate([direcciones, puntos], axis=1)

            # Concatenar imagenes reales y falsas 
            X = np.concatenate([image_batch, generated_images])
            
            # Etiquetas para datos generados y reales
            y_dis = np.zeros(2*batch_size)
            y_dis[:batch_size] = 0.9
            
            # Pre entrenamiento del discriminador sobre datos falsos y reales antes de entrenar la GAN 
            discriminator.trainable = True
            discriminator.train_on_batch(X, y_dis)
            
            #Tricking the noised input of the Generator as real data
            y_gen = np.ones(batch_size)
            
            # During the training of gan, 
            # the weights of discriminator should be fixed. 
            #We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            #training  the GAN by alternating the training of the Discriminator 
            #and training the chained GAN model with Discriminator’s weights freezed.
            gan.train_on_batch([gen_input, puntos], y_gen)
            
        #if (e == 1 or e % 20 == 0):
            #numpy.savetxt("content/Resultados/resultado%d.csv", np.concatenate([generator.predict(gen_input), puntos], delimiter=",")
    generator.save('my_model.h5')


training(1,1019)

# Guardar el modelo







  0%|          | 0/1019 [00:00<?, ?it/s]

Epoch 1







  0%|          | 1/1019 [00:08<2:23:06,  8.43s/it]




  0%|          | 2/1019 [00:09<1:43:58,  6.13s/it]




  0%|          | 3/1019 [00:09<1:16:26,  4.51s/it]




  0%|          | 4/1019 [00:10<57:10,  3.38s/it]  




  0%|          | 5/1019 [00:11<43:40,  2.58s/it]




  1%|          | 6/1019 [00:12<34:16,  2.03s/it]




  1%|          | 7/1019 [00:12<28:04,  1.66s/it]




  1%|          | 8/1019 [00:13<23:50,  1.41s/it]




  1%|          | 9/1019 [00:14<20:31,  1.22s/it]




  1%|          | 10/1019 [00:15<18:06,  1.08s/it]




  1%|          | 11/1019 [00:16<16:44,  1.00it/s]




  1%|          | 12/1019 [00:16<15:26,  1.09it/s]




  1%|▏         | 13/1019 [00:17<14:48,  1.13it/s]




  1%|▏         | 14/1019 [00:18<14:39,  1.14it/s]




  1%|▏         | 15/1019 [00:19<14:12,  1.18it/s]




  2%|▏         | 16/1019 [00:20<13:51,  1.21it/s]




  2%|▏         | 17/1019 [00:20<13:45,  1.21it/s]




  2%|▏         | 18/1019 [00:21<13:18,  1.25it/s]




  2%|▏         | 19/1019

In [0]:
ge = load_model("my_model.h5")

In [0]:
sol = ge.predict(gen_input)

In [58]:
sol

array([[-0.09110443,  0.149925  ,  0.06091851, ..., -0.13632783,
        -0.00057432,  0.31686336],
       [-0.05717064,  0.06986023, -0.00670128, ...,  0.01785408,
         0.06076445,  0.16998816],
       [-0.11535889,  0.08491795,  0.06946377, ..., -0.00993207,
         0.0983585 ,  0.27751586],
       ...,
       [ 0.03726367,  0.07920112, -0.06203206, ..., -0.03799467,
         0.07851568,  0.20775807],
       [-0.04226556,  0.11665943,  0.09695484, ..., -0.08059739,
         0.10210508,  0.181205  ],
       [ 0.03739544,  0.06477453,  0.08025398, ..., -0.00206206,
         0.00799904,  0.20520002]], dtype=float32)

###Postprocesado de datos de salida